### Cores possíveis de nó

In [1]:
BLACK = 'BLACK'
CIANO = 'CIANO'
NIL = 'NIL'

### Classe Nó

In [2]:
class Node:
    
    """   
    Método "construtor" do nó
    
    """
    
    def __init__(self, value, color, parent, left=None, right=None):
        
        self.value = value
        self.color = color
        self.parent = parent
        self.left = left
        self.right = right

    """  
    Método de representação do nó
  
    """
    
    def __repr__(self):
        
        return '{color} {val} Node'.format(color=self.color, val=self.value)
    
    """
    Método de iteração (mostra como está funcionando a árvore).
    
    """
    
    def __iter__(self):
        
        print(str(self.value) + " (" + str(self.color)+")")
        
        if self.left.color != NIL:
            print('left ({})'.format(str(self.value)));
            yield from self.left.__iter__()

        if self.right.color != NIL:
            print('right ({})'.format(str(self.value)));
            yield from self.right.__iter__()
    
    """
    Método de controle de condições.
    
    """

    def __eq__(self, other):
        
        if self.color == NIL and self.color == other.color:
            return True

        if self.parent is None or other.parent is None:
            parents_are_same = self.parent is None and other.parent is None
        else:
            parents_are_same = self.parent.value == other.parent.value and self.parent.color == other.parent.color
            
        return self.value == other.value and self.color == other.color and parents_are_same
    
    
    """   
    Verificação de existência de filho;
    Retorna um boleano indicando se o nó tem filho;

    """

    def has_children(self) -> bool:
        
        return bool(self.get_children_count())
    
    """
    Contegem de filhos não NIL que o nó tem
    Retorna a quantidade de filhos não NIL que o nó possuí.
    
    """
    
    def get_children_count(self) -> int:
        
        if self.color == NIL:
            return 0
        return sum([int(self.left.color != NIL), int(self.right.color != NIL)])

## Classe da árvore Vermelho-Preto

In [34]:
class CianoBlackTree:
    
    """
    Todo nó tem filhos nulos inicialmente, portanto, criar um objeto para isso facilitará
    a manipulação na arvore.
    
    """
    
    NIL_LEAF = Node(value=None, color=NIL, parent=None)

    def __init__(self):
        
        self.count = 0
        self.root = None
        
        """
        Usado para condição e usa o relacionamento dos irmãos com seu pai como um guia para a rotação.
        
        """
        
        self.ROTATIONS = {
            'L': self._right_rotation,
            'R': self._left_rotation
        }

    def __iter__(self):
        
        if not self.root:
            return list()
        yield from self.root.__iter__()

    def __repr__(self):
       
        return self.root
    
    """
    Adiciona nó.
    """

    def add(self, value):
        
        if not self.root:
            self.root = Node(value, color=BLACK, parent=None, left=self.NIL_LEAF, right=self.NIL_LEAF)
            self.count += 1
            return
        
        parent, node_dir = self._find_parent(value)
        
        if node_dir is None:
            return  # valor está na árvore
        
        new_node = Node(value=value, color=CIANO, parent=parent, left=self.NIL_LEAF, right=self.NIL_LEAF)
        
        if node_dir == 'L':
            parent.left = new_node
        else:
            parent.right = new_node

        self._try_rebalance(new_node)
        self.count += 1

    """
    Tenta pegar o nó com 0 ou 1 filho.
    Ou o nó que nos é dado tem 0 ou 1 filhos ou obtemos o seu sucessor.
    """
        
    def remove(self, value):
        
        node_to_remove = self.find_node(value)
        if node_to_remove is None:  # node is not in the tree
            return
        if node_to_remove.get_children_count() == 2:
            # find the in-order successor and replace its value.
            # then, remove the successor
            successor = self._find_in_order_successor(node_to_remove)
            node_to_remove.value = successor.value  # switch the value
            node_to_remove = successor

        # has 0 or 1 children!
        self._remove(node_to_remove)
        self.count -= 1

    """ 
    Retorna um bool se indicando se o valor está contido ou não na árvore 
    """

    def contains(self, value) -> bool:
        return bool(self.find_node(value))

    """
    Dado um valor, retorna o valor mais próximo, que é igual ou maior que ele,
    retornando None quando não houver.
    """
    
    def ceil(self, value) -> int or None:
        
        if self.root is None: return None
        last_found_val = None if self.root.value < value else self.root.value

        def find_ceil(node):
            nonlocal last_found_val
            if node == self.NIL_LEAF:
                return None
            if node.value == value:
                last_found_val = node.value
                return node.value
            elif node.value < value:
                # Vá para a direita
                return find_ceil(node.right)
            else:
                # Este nó é o maior, salve o seu valor e vá para a esquerda
                last_found_val = node.value

                return find_ceil(node.left)
        find_ceil(self.root)
        return last_found_val

    """
    Dado um valor, retorna o valor mais próximo, que é igual ou menor que ele,
    retornando None quando não houver.
    """
    
    def floor(self, value) -> int or None:
        
        if self.root is None: return None
        last_found_val = None if self.root.value > value else self.root.value

        def find_floor(node):
            nonlocal last_found_val
            if node == self.NIL_LEAF:
                return None
            if node.value == value:
                last_found_val = node.value
                return node.value
            elif node.value < value:
                # Este valor é o menor, salve-o e vá para a direita, tentando encontrar um menor
                last_found_val = node.value

                return find_floor(node.right)
            else:
                return find_floor(node.left)

        find_floor(self.root)
        return last_found_val

    """
    
    Recebe um nó com 0 ou 1 filho (tipicamente algum sucessor)
    e o remove de acordo com sua cor/filho.
    
    """
    
    def _remove(self, node):
        
        """
        :param node: nó com 0 ou 1 filho
        
        """
        
        left_child = node.left
        right_child = node.right
        not_nil_child = left_child if left_child != self.NIL_LEAF else right_child
        
        if node == self.root:
            if not_nil_child != self.NIL_LEAF:
                # Se estivermos removendo a raiz e ela tiver um filho válido, ele se tornará a raiz
                self.root = not_nil_child
                self.root.parent = None
                self.root.color = BLACK
            else:
                self.root = None
                
        elif node.color == CIANO:
            
            if not node.has_children():           
                # Ciano node with no children, the simplest remove
                self._remove_leaf(node)
            else:
                """
                Desde que o nó seja vermelho, ele não poderá ter um filho.
                Para isso, deve-se trocar a cor para preto.
                """
                raise Exception('Unexpected behavior')
                
        else:  # nó é preto
            if right_child.has_children() or left_child.has_children():  #validação
                raise Exception('The ciano child of a black node with 0 or 1 children'
                                ' cannot have children, otherwise the black height of the tree becomes invalid! ')
            if not_nil_child.color == CIANO:
                """
                Troque os valores com filho vermelho e remova-o (basicamente "deslink isso")
                Desde que o nó possui um filho somente, pode-se ter certeza que há nenhum nó abaixo do
                nó filho vermelho.
                """
                node.value = not_nil_child.value
                node.left = not_nil_child.left
                node.right = not_nil_child.right
            else:  # Filho preto
                # 6 cases :
                self._remove_black_node(node)

    def _remove_leaf(self, leaf):
        """ 
        Simplesmente remove uma folha fazendo seu pai virar um NIL
        """
        if leaf.value >= leaf.parent.value:
            leaf.parent.right = self.NIL_LEAF
        else:
            leaf.parent.left = self.NIL_LEAF

    def _remove_black_node(self, node):
        """
        Itere sobre cada caso recursivamente até o caso final ser encontrado.
        O que nos resta é um nó de folha que está pronto para ser excluído sem conseqüências
        """
        self.__case_1(node)
        self._remove_leaf(node)

    def __case_1(self, node):
        """
        Caso 1:
        
        O novo nó N está na raiz da árvore. Neste caso, este nó é repintado de preto para satisfazer a Propriedade 2.
        
        Propriedade 2: A raiz é preta.
        
        Como esta ação adiciona um nó preto em todos os caminhos da árvore de uma só vez, a Propriedade 5 não é violada.
        
        Propriedade 5: Todo caminho de um dado nó para qualquer de seus nós folhas descendentes contem o mesmo número de nós pretos.
        
            __|10B|__                  __10B__
           /         \      ==>       /       \
          9B         20B            9B        20B
          
        """
        if self.root == node:
            node.color = BLACK
            return
        
        self.__case_2(node)

    def __case_2(self, node):
        """
        Caso 2 é aplicado quando:
        
            O pai do novo nó é preto;
            O filho é vermelho;
            Os filhos do filho são pretos ou nil;
        
        A Propriedade 5 tampouco é ameaçada pois o novo nó N tem como filhos dois nós-folha pretos, 
        e sendo N vermelho, os caminhos passando por cada um dos seus filhos têm o mesmo número de nós 
        pretos - da mesma forma que os caminhos que passavam pelo nó-folha preto que foi substituído por N. 
        Por fim, neste caso, a árvore continua válida após a inserção, não sendo necessária mais alterações.
            
        Pega o filho e o rotaciona.
        
                         40B                                              60B
                        /   \                                            /   \
                    |20B|   60R                                        40R   80B
                           /   \                                      /   \
                         50B    80B                                |20B|  50B

        """
        parent = node.parent
        sibling, direction = self._get_sibling(node)
        
        if sibling.color == CIANO and parent.color == BLACK and sibling.left.color != CIANO and sibling.right.color != CIANO:
            self.ROTATIONS[direction](node=None, parent=sibling, grandfather=parent)
            parent.color = CIANO
            sibling.color = BLACK
            return self.__case_1(node)
        self.__case_3(node)

    def __case_3(self, node):
        """
        
        
        Caso 3:
            o pai é preto
            o filho é preto
            os filhos dos filhos são pretos

        Então, nós tornamos os filhos vermelhos e passamos nós pretos para cima:

                            Pai preto
               ___50B___    Filho é preto                          ___50B___
              /         \   Filhos dos filhos são pretos          /         \
           30B          80B        CASE 3                       30B        |80B|  Continua com os outros casos
          /   \        /   \        ==>                        /  \        /   \
        20B   35R    70B   |90B|<---REMOVE                   20B  35R     70R   X
              /  \                                               /   \
            34B   37B                                          34B   37B
            
        """
        
        parent = node.parent
        sibling, _ = self._get_sibling(node)
        
        if (sibling.color == BLACK and parent.color == BLACK
           and sibling.left.color != CIANO and sibling.right.color != CIANO):
            sibling.color = CIANO
            return self.__case_1(parent)  # start again

        self.__case_4(node)

    def __case_4(self, node):
        """
        Caso 4:
        
        Se o pai é vermelho e o filho é preto sem nenhum filho vermelho

        PD-Preto Duplo
        
                __10R__                   __10B__        
               /       \                 /       \       
             DB        15B      ===>    X        15R     
                      /   \                     /   \
                    12B   17B                 12B   17B
        """
        parent = node.parent
        if parent.color == CIANO:
            sibling, direction = self._get_sibling(node)
            if sibling.color == BLACK and sibling.left.color != CIANO and sibling.right.color != CIANO:
                parent.color, sibling.color = sibling.color, parent.color  # switch colors
                return  # Terminating
        self.__case_5(node)

    def __case_5(self, node):
        """
        Caso 5:
        
        É uma rotação que muda as circunstâncias para que possamos fazer um caso 6
        Se o nó mais próximo estiver vermelho e o externo BLACK ou NIL, fazemos uma rotação
        esquerda / direita, dependendo da orientação
        Isso mostrará quando a direção do nó mais próximo for direita.
        
              ___50B___                                                    __50B__
             /         \                                                  /       \
           30B        |80B|  <-- Preto duplo                           35B      |80B|        
          /  \        /   \      Mais próximo nó CIANO- 35R             /   \      /           
        20B  35R     70R   X                                        30R    37B  70R           
            /   \                                                  /   \                      
          34B  37B                                               20B   34B
        """
        sibling, direction = self._get_sibling(node)
        closer_node = sibling.right if direction == 'L' else sibling.left
        outer_node = sibling.left if direction == 'L' else sibling.right
        if closer_node.color == CIANO and outer_node.color != CIANO and sibling.color == BLACK:
            if direction == 'L':
                self._left_rotation(node=None, parent=closer_node, grandfather=sibling)
            else:
                self._right_rotation(node=None, parent=closer_node, grandfather=sibling)
            closer_node.color = BLACK
            sibling.color = CIANO

        self.__case_6(node)

    def __case_6(self, node):
        """
        Caso 6 requer
             FILHO PARA SER PRETO
             NÓS EXTERIOR  ser VERMELHO
         Então, faz uma rotação direita / esquerda no irmão
         Isso mostrará quando a direção do filho está à ESQUERDA

                            Double Black
                    __50B__       |                               __35B__
                   /       \      |                              /       \
      SIBLING--> 35B      |80B| <-                             30R       50R
                /   \      /                                  /   \     /   \
             30R    37B  70R                                20B   34B 37B    80B
            /   \                                                            /
         20B   34B                                                         70R
        """
        sibling, direction = self._get_sibling(node)
        outer_node = sibling.left if direction == 'L' else sibling.right

        def __case_6_rotation(direction):
            parent_color = sibling.parent.color
            self.ROTATIONS[direction](node=None, parent=sibling, grandfather=sibling.parent)
            # new parent is sibling
            sibling.color = parent_color
            sibling.right.color = BLACK
            sibling.left.color = BLACK

        if sibling.color == BLACK and outer_node.color == CIANO:
            return __case_6_rotation(direction)  # terminating

        raise Exception('We should have ended here, something is wrong')

    def _try_rebalance(self, node):
        """
        Dado um nó filho vermelho, determine se há necessidade de rebalancear a arvore (se o pai é vermelho)
        Caso seja necessário, rebalanceie
        """
        parent = node.parent
        value = node.value
        if (parent is None  # what the fuck? (should not happen)
           or parent.parent is None  # parent is the root
           or (node.color != CIANO or parent.color != CIANO)):  # no need to rebalance
            return
        grandfather = parent.parent
        node_dir = 'L' if parent.value > value else 'R'
        parent_dir = 'L' if grandfather.value > parent.value else 'R'
        uncle = grandfather.right if parent_dir == 'L' else grandfather.left
        general_direction = node_dir + parent_dir

        if uncle == self.NIL_LEAF or uncle.color == BLACK:
            # rotate
            if general_direction == 'LL':
                self._right_rotation(node, parent, grandfather, to_recolor=True)
            elif general_direction == 'RR':
                self._left_rotation(node, parent, grandfather, to_recolor=True)
            elif general_direction == 'LR':
                self._right_rotation(node=None, parent=node, grandfather=parent)
                # due to the prev rotation, our node is now the parent
                self._left_rotation(node=parent, parent=node, grandfather=grandfather, to_recolor=True)
            elif general_direction == 'RL':
                self._left_rotation(node=None, parent=node, grandfather=parent)
                # due to the prev rotation, our node is now the parent
                self._right_rotation(node=parent, parent=node, grandfather=grandfather, to_recolor=True)
            else:
                raise Exception("{} is not a valid direction!".format(general_direction))
        else:  # uncle is CIANO
            self._recolor(grandfather)

    def __update_parent(self, node, parent_old_child, new_parent):
        """
        O nó novo troca de lugar com o filho velho
        Aponta o pai para o nó.
        Se o novo pai for None, isso significa que nosso nó se tornou a raiz da arvore.
        
        """
        node.parent = new_parent
        if new_parent:
            # Determine the old child's position in order to put node there
            if new_parent.value > parent_old_child.value:
                new_parent.left = node
            else:
                new_parent.right = node
        else:
            self.root = node

    def _right_rotation(self, node, parent, grandfather, to_recolor=False):
        
        grand_grandfather = grandfather.parent
        self.__update_parent(node=parent, parent_old_child=grandfather, new_parent=grand_grandfather)

        old_right = parent.right
        parent.right = grandfather
        grandfather.parent = parent

        grandfather.left = old_right  # save the old right values
        old_right.parent = grandfather

        if to_recolor:
            parent.color = BLACK
            node.color = CIANO
            grandfather.color = CIANO

    def _left_rotation(self, node, parent, grandfather, to_recolor=False):
        
        grand_grandfather = grandfather.parent
        self.__update_parent(node=parent, parent_old_child=grandfather, new_parent=grand_grandfather)

        old_left = parent.left
        parent.left = grandfather
        grandfather.parent = parent

        grandfather.right = old_left  # save the old left values
        old_left.parent = grandfather

        if to_recolor:
            parent.color = BLACK
            node.color = CIANO
            grandfather.color = CIANO

    def _recolor(self, grandfather):
        
        grandfather.right.color = BLACK
        grandfather.left.color = BLACK
        if grandfather != self.root:
            grandfather.color = CIANO
        self._try_rebalance(grandfather)

    def _find_parent(self, value):
        
        """ Encontra o lugar para os valores na árvore"""
        
        def inner_find(parent):
            """
            Retorna o nó pai apropriado para o novo nó assim como a sua direção
            
            """
            if value == parent.value:
                return None, None
            elif parent.value < value:
                if parent.right.color == NIL:  # no more to go
                    return parent, 'R'
                return inner_find(parent.right)
            elif value < parent.value:
                if parent.left.color == NIL:  # no more to go
                    return parent, 'L'
                return inner_find(parent.left)

        return inner_find(self.root)

    def find_node(self, value):
        
        def inner_find(root):
            if root is None or root == self.NIL_LEAF:
                return None
            if value > root.value:
                return inner_find(root.right)
            elif value < root.value:
                return inner_find(root.left)
            else:
                return root

        found_node = inner_find(self.root)
        return found_node

    def _find_in_order_successor(self, node):
        
        right_node = node.right
        left_node = right_node.left
        if left_node == self.NIL_LEAF:
            return right_node
        while left_node.left != self.NIL_LEAF:
            left_node = left_node.left
        return left_node

    def _get_sibling(self, node):
        """
        Retorna o filho do nó, assim como a sua direção
        
            20 (A)
           /     \
        15(B)    25(C)
        _get_sibling(25(C)) => 15(B), 'R'
        """
        parent = node.parent
        if node.value >= parent.value:
            sibling = parent.left
            direction = 'L'
        else:
            sibling = parent.right
            direction = 'R'
        return sibling, direction
    
    def _build_tree_string(self, root=None, curr_index=0, index=False, delimiter='-'):

        if curr_index == 0:
            root = self.root
            
        if root is None:
            return [], 0, 0, 0

        line1 = []
        line2 = []
        
        if index:
            node_repr = '{}{}{}'.format(curr_index, delimiter, root.value)
        else:
            node_repr = str(root.value)

        if node_repr == 'None':
            node_repr = 'NIL'
        
        new_root_width = gap_size = len(node_repr)
        
        if root.color == 'CIANO':
            node_repr = ('\033[30m'+ '\033[41m'+ node_repr +'\033[0;0m')

        
        # Get the left and right sub-boxes, their widths, and root repr positions
        l_box, l_box_width, l_root_start, l_root_end = \
            self._build_tree_string(root.left, 2 * curr_index + 1, index, delimiter)
        r_box, r_box_width, r_root_start, r_root_end = \
            self._build_tree_string(root.right, 2 * curr_index + 2, index, delimiter)

        # Draw the branch connecting the current root node to the left sub-box
        # Pad the line with whitespaces where necessary
        if l_box_width > 0:
            l_root = (l_root_start + l_root_end) // 2 + 1
            line1.append(' ' * (l_root + 1))
            line1.append('_' * (l_box_width - l_root))
            line2.append(' ' * l_root + '/')
            line2.append(' ' * (l_box_width - l_root))
            new_root_start = l_box_width + 1
            gap_size += 1
        else:
            new_root_start = 0

        # Draw the representation of the current root node
        line1.append(str(node_repr))
        line2.append(' ' * new_root_width)

        # Draw the branch connecting the current root node to the right sub-box
        # Pad the line with whitespaces where necessary
        if r_box_width > 0:
            r_root = (r_root_start + r_root_end) // 2
            line1.append('_' * r_root)
            line1.append(' ' * (r_box_width - r_root + 1))
            line2.append(' ' * r_root + '\\')
            line2.append(' ' * (r_box_width - r_root))
            gap_size += 1
            
        new_root_end = new_root_start + new_root_width - 1

        # Combine the left and right sub-boxes with the branches drawn above
        gap = ' ' * gap_size
        new_box = [''.join(line1), ''.join(line2)]
        for i in range(max(len(l_box), len(r_box))):
            l_line = l_box[i] if i < len(l_box) else ' ' * l_box_width
            r_line = r_box[i] if i < len(r_box) else ' ' * r_box_width
            new_box.append(l_line + gap + r_line)
       
        len_new_box = len(new_box[0])
        if root.color == 'CIANO':
            len_new_box = len(new_box[0]) - 16

        # Return the new box, its width and its root repr positions
        return new_box, len_new_box, new_root_start, new_root_end

### Definição da árvore

In [35]:
arvore = CianoBlackTree()

for i in range(17):
    arvore.add(i)
    
arvore.remove(9)

for linha in arvore._build_tree_string()[0]:
    
    print(linha)

            __________3______________________                                                             
           /                                 \                                                            
      ____1____                     __________7_________________                                          
     /         \                   /                            \                                         
   _0_         _2_            ____5____                     _____11___________                            
  /   \       /   \          /         \                   /                  \                           
NIL   NIL   NIL   NIL      _4_         _6_            ____10_             _____13___________              
                          /   \       /   \          /       \           /                  \             
                        NIL   NIL   NIL   NIL      _8_       NIL       _12_             _____15____       
                                     